# Sistema de Recomendação - Filtragem Colaborativa
Recomendando relatórios aos usuários

A filtragem colaborativa baseada em itens calcula predições baseado nos padrões de avaliações entre itens, ou seja, considera o conjunto de itens que o usuário avaliou previamente, e computa quão similares são, em relação a um item alvo.

### Parte I: Leitura da base de dados
A base de dados neste exemplo consiste em 2 consultas que foram extraídas diretamente do banco de dados e salvos em formato csv, com aproximadamente 1.6 milhões de linhas de logs. Serão utilizados as bibliotecas, pandas, numpy e sklearn.metrics.pairwise. Os dados serão carregados utilizando o read_csv() da biblioteca pandas. As comparações serão realizadas utilizando o pacote de métricas cosine_similarity da biblioteca sklearn, que compara duas matriz numpy.

In [1]:
# Importando as bibliotecas  
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
# Importando os dados e armazenando em DataFrames
# logs1 = 1 semestre de 2021
# logs2 = 2 semestre de 2021

logs1 = pd.read_csv("/Users/lucasmartins/Documents/logs_1semestre2021.csv")
logs2 = pd.read_csv("/Users/lucasmartins/Documents/logs_2semestre2021.csv")

In [3]:
# Concatenando os DataFrames
item_ratings = pd.concat([logs1,logs2]).reset_index(drop=True)

# Inspecionando o DataFrame
item_ratings

,Id,user_group_id,response_name
0,219,7,"{livre, metaLoja}"
1,219,7,"{livre, metaLoja}"
2,219,7,"{livre, metaLoja}"
3,219,7,"{venda, VendaNovaclusterLojas}"
4,219,7,"{venda, vendaNovaLoja}"
...,...,...,...
1615858,673,6,"{InteligenciaVarejo, vendaAnaliticaMaterial}"
1615859,673,6,"{InteligenciaVarejo, vendaAnaliticaMaterial}"
1615860,431,6,"{InteligenciaVarejo, CriticaProdutosLinhaEcomm..."
1615861,94,5,"{InteligenciaVarejo, CriticaProdutosLinhaEcomm..."


### Parte II: Pré-processamento de dados
Conforme observado na leitura da base de dados , os dados vindos de produção não estão totalmente normatizados, necessitando de um pré-processamento e limpeza antes de serem usados no algoritmo. Devido ao volume dos dados, as extrações foram feitas separadamente, primeiramente os DataFrames serão concatenados em um DataFrame único, facilitando o manuseio e análise dos dados.

In [4]:
# Removendo colunas desnecessárias
item_ratings.drop(["user_group_id"], axis=1, inplace=True)

# Inspecionando o DataFrame
item_ratings

,Id,response_name
0,219,"{livre, metaLoja}"
1,219,"{livre, metaLoja}"
2,219,"{livre, metaLoja}"
3,219,"{venda, VendaNovaclusterLojas}"
4,219,"{venda, vendaNovaLoja}"
...,...,...
1615858,673,"{InteligenciaVarejo, vendaAnaliticaMaterial}"
1615859,673,"{InteligenciaVarejo, vendaAnaliticaMaterial}"
1615860,431,"{InteligenciaVarejo, CriticaProdutosLinhaEcomm..."
1615861,94,"{InteligenciaVarejo, CriticaProdutosLinhaEcomm..."


In [5]:
# Renomeando as colunas 
item_ratings.rename(columns={'Id': 'UserId', 'response_name': 'Pergunta'}, inplace=True)

# Inspecionando o DataFrame
item_ratings

,UserId,Pergunta
0,219,"{livre, metaLoja}"
1,219,"{livre, metaLoja}"
2,219,"{livre, metaLoja}"
3,219,"{venda, VendaNovaclusterLojas}"
4,219,"{venda, vendaNovaLoja}"
...,...,...
1615858,673,"{InteligenciaVarejo, vendaAnaliticaMaterial}"
1615859,673,"{InteligenciaVarejo, vendaAnaliticaMaterial}"
1615860,431,"{InteligenciaVarejo, CriticaProdutosLinhaEcomm..."
1615861,94,"{InteligenciaVarejo, CriticaProdutosLinhaEcomm..."


In [6]:
# Resumo dos dados
item_ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1615863 entries, 0 to 1615862
Data columns (total 2 columns):
 #   Column    Non-Null Count    Dtype 
---  ------    --------------    ----- 
 0   UserId    1615863 non-null  int64 
 1   Pergunta  1615863 non-null  object
dtypes: int64(1), object(1)
memory usage: 24.7+ MB


In [7]:
# Agrupando e contando as perguntas para que seja gerado a coluna de “Avaliação”.
# Através dessa coluna que encontraremos os usuários que possuem preferências semelhantes.

# Agrupando as colunas "UserId" e "Pergunta, contando as ocorrências e armazenando na coluna "Avaliação"
item_ratings = item_ratings.groupby(['UserId','Pergunta']).size().reset_index().rename(columns={0:'Avaliacao'})

# Classificando de forma decrescente
item_ratings = item_ratings.sort_values(by="Avaliacao", ascending=False)

# Inspecionando o DataFrame
item_ratings

,UserId,Pergunta,Avaliacao
13385,541,"{compras, Price_VencimentoProduto}",19976
5413,172,"{compras, LojaAVencer}",16824
12635,520,"{compras, Price_VencimentoProduto}",13611
13185,536,"{compras, Price_VencimentoProduto}",13571
5422,172,"{compras, Price_VencimentoProduto}",12304
...,...,...,...
3671,112,"{admin, AnaliseConcorrentes}",1
3672,112,"{admin, negociacoes}",1
3674,112,"{compras, AvaliacaoLoja}",1
3677,112,"{compras, Dnv}",1


### Parte III: Matriz de classificação
Como os dados estão em um DataFrame, o método pivot do pandas pode ser usado para modelar os dados em torno das colunas especificadas. Queremos perguntas como o índice, as colunas representando os usuários e a avaliação como os valores correspondentes.
 
Será transformado em uma matriz de classificação, onde cada linha representa uma pergunta e cada coluna representa um usuário.
A primeira coisa que pode se tornar aparente após a transformação é o número de entradas ausentes, demonstrado pelos valores NaN. Isso é esperado, um usuário raramente terá avaliado todas perguntas, e é igualmente raro que uma pergunta tenha sido avaliada por todos os usuários. Isso é um problema, pois a maioria das métricas de similaridade não lida muito bem com dados ausentes.

In [8]:
# Transformando o DataFrame em uma matriz de classificação de usuários onde cada linha representa uma pergunta e cada coluna representa um usuário.

# Transformando o DataFrame
item_ratings_pivot = item_ratings.pivot_table(index="Pergunta", columns="UserId", values="Avaliacao")

# Inspecionando a tabela transformada
item_ratings_pivot

UserId,3,8,9,10,12,13,14,15,19,20,...,735,736,737,738,740,742,744,745,746,747
Pergunta,,,,,,,,,,,,,,,,,,,,,
"{ExclusivoContasReceber, ConsultaClientePDV}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"{ExclusivoEcommerce, Status Pedido Ecommerce}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"{ExclusivoEcommerce, estadoOperacao}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"{ExclusivoRH, ConsultaFuncionarioPDV}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"{InteligenciaVarejo, AlteraCompradorEstruturaMercadologica}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"{venda, vendaRestauranteSecao}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"{venda, vendaRestaurante}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"{venda, vendaSecProduto}",NaN,NaN,1.0,40.0,6.0,NaN,5.0,3.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
# Nessa etapa será preenchido os dados ausentes com informações que não devem influenciar os dados da análise. 
# Será obtido a pontuação média de cada usuário referente todas as perguntas e, em seguida, uma pontuação neutra
# para centralizar as pontuações dos usuários em torno de zero, que será uma pontuação neutra, minimizando o impacto
# em seu perfil geral, permitindo a comparação dos usuários.

# Obtendo a classificação média para cada pergunta 
avg_ratings = item_ratings_pivot.mean(axis=1)

# Centralizando as avaliações em torno de 0
item_ratings_table_centered = item_ratings_pivot.sub(avg_ratings, axis=0)

# Preenchendo os dados com 0s
item_ratings_table_normed = item_ratings_table_centered.fillna(0)

# Inspecionando a tabela normalizada
item_ratings_table_normed

UserId,3,8,9,10,12,13,14,15,19,20,...,735,736,737,738,740,742,744,745,746,747
Pergunta,,,,,,,,,,,,,,,,,,,,,
"{ExclusivoContasReceber, ConsultaClientePDV}",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
"{ExclusivoEcommerce, Status Pedido Ecommerce}",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
"{ExclusivoEcommerce, estadoOperacao}",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
"{ExclusivoRH, ConsultaFuncionarioPDV}",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
"{InteligenciaVarejo, AlteraCompradorEstruturaMercadologica}",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"{venda, vendaRestauranteSecao}",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
"{venda, vendaRestaurante}",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
"{venda, vendaSecProduto}",0.000000,0.000000,-14.007143,24.992857,-9.007143,0.000000,-10.007143,-12.007143,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000


### Parte IV: Matriz de similaridade
Expressando a similaridade entre perguntas

Utilizando o cosseno similaridade, nota que por representar o cosseno entre vetores, o momento em que os vetores de avaliação estão mais próximos, demonstrando máxima similaridade entre si, esta quando formam entre si um ângulo de 0o, isto é, quando seu cosseno tem valor 1.
Na mesma lógica, apresentam maior rejeição entre si quando formam um ângulo de 180o, resultando em um cosseno de -1.


In [10]:
#  Vemos então que cosseno similaridade gera os valores no intervalo [-1,1],
#  tal que -1 representa a máxima incompatibilidade entre as perguntas e 1 sendo a máxima compatibilidade.

# Obtendo as perguntas que serão comparadas
perg_1 = item_ratings_table_normed.loc['{venda, vendaNovaLoja}', :].values.reshape(1, -1)
perg_2 = item_ratings_table_normed.loc['{livre, painelVenda}', :].values.reshape(1, -1)

# Transformando em uma matriz NumPy e expressando a similaridade
similarity_A = cosine_similarity(perg_1, perg_2)

# Inspecionando a similaridade
print(similarity_A)

[[0.65488918]]


In [11]:
# Gerando a matriz de similaridade

# Para fazer as recomendações é necessário encontrar as perguntas mais semelhantes em geral.
# Para isso, precisamos encontrar as semelhanças entre todos de uma só vez, resultando em uma matriz de similaridade
# entre todas as perguntas. Com essa matriz calculada, poderemos fazer as recomendações.


similarities = cosine_similarity(item_ratings_table_normed)

# Envolvendo as semelhanças em um DataFrame
cosine_similarity_df = pd.DataFrame(similarities, index=item_ratings_table_normed.index, columns=item_ratings_table_normed.index)

# Matriz de similaridade entre todas as perguntas
cosine_similarity_df.head()

Pergunta,"{ExclusivoContasReceber, ConsultaClientePDV}","{ExclusivoEcommerce, Status Pedido Ecommerce}","{ExclusivoEcommerce, estadoOperacao}","{ExclusivoRH, ConsultaFuncionarioPDV}","{InteligenciaVarejo, AlteraCompradorEstruturaMercadologica}","{InteligenciaVarejo, CheckLinhaOrbis}","{InteligenciaVarejo, CheckListLojaInput}","{InteligenciaVarejo, CheckListLoja}","{InteligenciaVarejo, CheckPrecoDiferenteSTM}","{InteligenciaVarejo, CriticaForaLinhaSemOferta}",...,"{venda, vendaNovaSecao}","{venda, vendaPorTempo}","{venda, vendaRestauranteCategoria}","{venda, vendaRestauranteDepartamento}","{venda, vendaRestauranteGrupo}","{venda, vendaRestauranteSecao}","{venda, vendaRestaurante}","{venda, vendaSecProduto}","{{_looqbox}, noInterpretation}","{{_looqbox}, suggestion}"
Pergunta,,,,,,,,,,,,,,,,,,,,,
"{ExclusivoContasReceber, ConsultaClientePDV}",1.000000,0.0,-0.118618,0.746591,-0.131782,0.0,0.023787,0.011187,0.047607,0.065712,...,-0.021784,-0.077764,0.0,0.044354,0.0,-0.584347,0.0,-0.003068,-0.040955,0.063739
"{ExclusivoEcommerce, Status Pedido Ecommerce}",0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000
"{ExclusivoEcommerce, estadoOperacao}",-0.118618,0.0,1.000000,-0.092249,0.000000,0.0,0.000000,0.015731,0.000000,0.000000,...,-0.024267,-0.020388,0.0,0.000000,0.0,0.182574,0.0,-0.044361,-0.060335,-0.099797
"{ExclusivoRH, ConsultaFuncionarioPDV}",0.746591,0.0,-0.092249,1.000000,-0.193719,0.0,0.008605,-0.000267,0.033722,0.232509,...,-0.043771,-0.058852,0.0,0.018125,0.0,-0.392111,0.0,-0.024565,-0.052057,0.003924
"{InteligenciaVarejo, AlteraCompradorEstruturaMercadologica}",-0.131782,0.0,0.000000,-0.193719,1.000000,0.0,-0.026050,-0.094017,0.108799,0.069244,...,-0.011975,0.026893,0.0,-0.095590,0.0,-0.144338,0.0,-0.004934,0.010749,-0.004584


### Parte V: Recomendando
Nessa etapa faremos as recomendações em relação a um pergunta alvo.

In [12]:
# Encontrando os valores de similaridade para uma pergunta específica
cosine_similarity_series = cosine_similarity_df.loc['{venda, vendaNovaLoja}']

# Ordenando os valores do maior para o menor
ordered_similarities = cosine_similarity_series.sort_values(ascending=False)

# Fazendo as recomendações
ordered_similarities.head(10)

Pergunta
{venda, vendaNovaLoja}                  1.000000
{livre, painelVenda}                    0.654889
{venda, VendaNovaclusterLojas}          0.613398
{venda, vendaNovaDepartamento}          0.559896
{livre, painelDash}                     0.445001
{compras, dashVendas}                   0.408812
{compras, dashVendasPorLoja}            0.404733
{venda, vendaNovaSecao}                 0.399738
{venda, vendaNovaGrupo}                 0.370622
{compras, dashVendasPorDepartamento}    0.340848
Name: {venda, vendaNovaLoja}, dtype: float64

In [13]:
# Encontrando os valores de similaridade para uma pergunta específica
cosine_similarity_series = cosine_similarity_df.loc['{InteligenciaVarejo, vendaLojaMaterialEstoque}']

# Ordenando os valores do maior para o menor
ordered_similarities = cosine_similarity_series.sort_values(ascending=False)

# Fazendo as recomendações
ordered_similarities.head(10)

Pergunta
{InteligenciaVarejo, vendaLojaMaterialEstoque}    1.000000
{compras, dashVendasPorProduto}                   0.760234
{venda, vendaLojaProduto}                         0.682742
{venda, vendaSecProduto}                          0.639058
{compras, dashVendasPorSecao}                     0.462461
{compras, dashVendasPorLoja}                      0.429454
{livre, manifestoMarche}                          0.399099
{venda, vendaLojaProdutoEstoque}                  0.368531
{livre, painelDash}                               0.337421
{compras, dashVendasPorDepartamento}              0.314054
Name: {InteligenciaVarejo, vendaLojaMaterialEstoque}, dtype: float64

In [14]:
# Encontrando os valores de similaridade para uma pergunta específica
cosine_similarity_series = cosine_similarity_df.loc['{produto, infoImposto}']

# Ordenando os valores do maior para o menor
ordered_similarities = cosine_similarity_series.sort_values(ascending=False)

# Fazendo as recomendações
ordered_similarities.head(10)

Pergunta
{produto, infoImposto}                      1.000000
{fornecedor, fornecedorSellout}             0.592470
{InteligenciaVarejo, MaterialForn}          0.437749
{compras, VendaRappiPlu}                    0.414246
{fornecedor, fornecedorEncontrar}           0.334695
{compras, infoCompradores}                  0.248918
{fornecedor, fornecedorLinha}               0.233263
{produto, produtoSellout}                   0.233186
{compras, departamentoSellout}              0.211521
{InteligenciaVarejo, VendaPorFornecedor}    0.178727
Name: {produto, infoImposto}, dtype: float64

In [15]:
# Encontrando os valores de similaridade para uma pergunta específica
cosine_similarity_series = cosine_similarity_df.loc['{auditoria, auditoriaBaseVendaLojaSemCMV}']

# Ordenando os valores do maior para o menor
ordered_similarities = cosine_similarity_series.sort_values(ascending=False)

# Fazendo as recomendações
ordered_similarities.head(10)

Pergunta
{auditoria, auditoriaBaseVendaLojaSemCMV}     1.000000
{livre, painelAuditoria}                      0.871797
{admin, looqboxUsers}                         0.862583
{auditoria, auditoriaBaseVendaRestaurante}    0.827769
{livre, AuditoriaZM}                          0.650364
{livre, SaldoVoucher}                         0.531197
{livre, ConsultaVoucher}                      0.525402
{compras, CargaRotinas}                       0.293702
{admin, JobsSql}                              0.286501
{compras, VendasPorFinalizadoraOnOFF}         0.278119
Name: {auditoria, auditoriaBaseVendaLojaSemCMV}, dtype: float64

In [16]:
# Encontrando os valores de similaridade para uma pergunta específica
cosine_similarity_series = cosine_similarity_df.loc['{livre, infoMaterial}']

# Ordenando os valores do maior para o menor
ordered_similarities = cosine_similarity_series.sort_values(ascending=False)

# Fazendo as recomendações
ordered_similarities.head(10)

Pergunta
{livre, infoMaterial}                 1.000000
{livre, infoProduto}                  0.545769
{livre, precoProduto}                 0.407016
{compras, aVencerLoja}                0.395929
{compras, Dnv}                        0.355300
{compras, ConsultaCashBackCpf}        0.349057
{livre, AnaliseCliente}               0.348198
{compras, Price_VencimentoProduto}    0.345783
{livre, CuponsAniversariantes}        0.318191
{{_looqbox}, noInterpretation}        0.309612
Name: {livre, infoMaterial}, dtype: float64

In [17]:
# Encontrando os valores de similaridade para uma pergunta específica
cosine_similarity_series = cosine_similarity_df.loc['{gcat, EstruturaMercadologica}']

# Ordenando os valores do maior para o menor
ordered_similarities = cosine_similarity_series.sort_values(ascending=False)

# Fazendo as recomendações
ordered_similarities.head(10)

Pergunta
{gcat, EstruturaMercadologica}                    1.000000
{compras, ItensObrigatorioRappi}                  0.562995
{compras, ListaProdutosObrigatorioRappi}          0.531126
{Rappi, RegraFiltroRappi}                         0.331868
{InteligenciaVarejo, PainelEtiquetasLojas}        0.301316
{InteligenciaVarejo, PainelEtiquetasComprador}    0.253305
{ExclusivoRH, ConsultaFuncionarioPDV}             0.195414
{compras, PenetracaoPorDepartamento}              0.185994
{compras, infoCompradores}                        0.177508
{compras, estoquenegativo}                        0.177508
Name: {gcat, EstruturaMercadologica}, dtype: float64

In [18]:
# Encontrando os valores de similaridade para uma pergunta específica
cosine_similarity_series = cosine_similarity_df.loc['{ExclusivoRH, ConsultaFuncionarioPDV}']

# Ordenando os valores do maior para o menor
ordered_similarities = cosine_similarity_series.sort_values(ascending=False)

# Fazendo as recomendações
ordered_similarities.head(10)

Pergunta
{ExclusivoRH, ConsultaFuncionarioPDV}              1.000000
{ExclusivoContasReceber, ConsultaClientePDV}       0.746591
{compras, infoCompradores}                         0.400479
{InteligenciaVarejo, PainelEtiquetasComprador}     0.395290
{compras, InformacaoRotinas}                       0.326500
{admin, logLooqboxByResponse}                      0.297108
{compras, vendaFinalizadora}                       0.235700
{InteligenciaVarejo, CriticaForaLinhaSemOferta}    0.232509
{compras, TotalVendasDelivery}                     0.209943
{gcat, EstruturaMercadologica}                     0.195414
Name: {ExclusivoRH, ConsultaFuncionarioPDV}, dtype: float64